In [1]:
import tensorflow as tf
import cv2

import numpy as np
import os

/home/james/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def load_image(img_name):
    img_path = os.path.join(test_imgs_dir,img_name)
    image = cv2.imread(img_path)
    image = np.expand_dims(image,axis=0)
    return image

In [3]:
# cv2.getTextSize
#cv2.rectangle (thickness = -1) -> green
#cv2.putText --> white

#Or different color for different class

In [4]:
def draw_bounding_box(img,detections,boxes,classes):
    img_height,img_width,_ = img.shape
    
    for i in detections:
        #Draw bounding box 
        ymin, xmin, ymax, xmax = boxes[i]
        
        xmin = int(xmin*img_width)
        xmax = int(xmax*img_width)
        ymin = int(ymin*img_height)
        ymax = int(ymax*img_height)
        
        
        #print("x goes from {0} to {1}".format(xmin,xmax))
        #print("y goes from {0} to {1}".format(ymin,ymax))
        
        
        cv2.rectangle(img,(xmin,ymin),(xmax,ymax),(0,0,255),thickness = 1)
        
        font = cv2.FONT_HERSHEY_SIMPLEX

        size = cv2.getTextSize(class_map[int(classes[i])],cv2.FONT_HERSHEY_SIMPLEX,0.5,1)

        text_width,text_height = size[0]
        
        cv2.rectangle(img,(xmin,ymin),(xmin+text_width,ymin-text_height),(0,0,255),thickness = -1)

        #cv2.getTextSize()
        cv2.putText(img,class_map[int(classes[i])],(xmin,ymin), font, 0.5,(255,255,255),1,cv2.LINE_AA)

In [5]:
#Parameters
threshold_score = 0.20

In [6]:
#Relative Paths...

path_to_infer_graph = "model/frozen_inference_graph.pb"
class_label_map_path = "model/mscoco_label_map.pbtxt"
test_imgs_dir = 'Test_Images'
result_imgs_dir = 'Results'
class_map_file = 'model/class_names.csv'

In [7]:
with open(class_map_file,'r') as csv_file:
    class_id_list = csv_file.readlines()

class_map = {}
for id_name in class_id_list:
    id_,name = id_name.strip().split(",")
    class_map[int(id_)] = name

In [8]:
detection_graph = tf.Graph()

In [9]:
img_names = [fname for fname in os.listdir(test_imgs_dir) if fname.lower().endswith(".png") or fname.lower().endswith(".jpg")]

In [10]:
img_names.sort()

In [11]:
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(path_to_infer_graph,mode='rb') as graph_file:
        serialized_graph = graph_file.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def)

In [12]:
sess = tf.InteractiveSession(graph = detection_graph)

In [13]:
# names = [n.name for n in sess.graph.as_graph_def().node]

In [14]:
for img_name in img_names:
    test_img = load_image(img_name)
    
    input_ = sess.graph.get_tensor_by_name("import/image_tensor:0")
    boxes = sess.graph.get_tensor_by_name("import/detection_boxes:0")
    scores = sess.graph.get_tensor_by_name("import/detection_scores:0")
    classes = sess.graph.get_tensor_by_name("import/detection_classes:0")
    num_detections = sess.graph.get_tensor_by_name("import/num_detections:0")
    (boxes, scores, classes, num_detections) = sess.run( 
                    [boxes, scores, classes, num_detections],
                    feed_dict={input_: test_img})
    
    boxes = np.squeeze(boxes,axis=0)
    scores = np.squeeze(scores,axis=0)
    classes = np.squeeze(classes,axis=0)
    test_img = np.squeeze(test_img,axis=0)
    
    detections = []
    for i,score in enumerate(scores):
        if score >= threshold_score:
            detections.append(i)
    
    draw_bounding_box(test_img,detections,boxes,classes)
    
    cv2.imwrite(os.path.join(result_imgs_dir,img_name),test_img)